# Toxicity Detection Model

Get live data with `get_comments.py` and transform Kaggle data set with `transform_train.py`

Get libraries by running `pip install -r requirements.txt`

This is a deep learning model.

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import tensorflow as tf
import keras
from keras import layers, Model
from keras.layers import Input, Bidirectional, Dense, Embedding, LSTM, Dropout
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import STOPWORDS, wordcloud
import re
from fuzzywuzzy import fuzz, process
from nltk.corpus import stopwords
import nltk

from tqdm import tqdm
import math

# Load Dataset

In [ ]:
!gdown 1MtOJQuJqiOHUKS8SvgQAfuTamwI5nagp

In [ ]:
df_train = pd.read_csv('new_train.csv')

### Data Cleansing
<ul>
    <li>Removal of special characters</li>
    <li>Expanding contractions</li>
    <li>Removal of stopping words</li>
    <li>Lowering text</li>
    <li>Replacing Obfuscated Profane Words</li>
</ul>

In [ ]:
def decontracted(phrase):
    '''
    This function decontracts words like won't to will not
    '''

    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

In [ ]:
def removeNonPrintable(com):
    com = com.replace('\\r', ' ')
    com = com.replace('\\n', ' ')
    com = com.replace('\\t', ' ')
    com = com.replace('\\"', ' ')
    return com

In [ ]:
def getUniqueWords(comments):
    unique_words = set()
    for comment in tqdm(comments):
        words = comment.split(" ")
        for word in words:
            if len(word) > 2:
                unique_words.add(word)
    
    return unique_words

In [ ]:
def getProfaneWords():
    profane_words = []
    with open("bad-words.txt","r") as f:
        for word in f:
            word = word.replace("\n","")
            profane_words.append(word)
    return profane_words
    

In [ ]:
def createMappingDict(profane_words, unique_words):
    # mapping dictionary
    mapping_dict = dict()
    
    # looping through each profane word
    for profane in tqdm(profane_words):
        mapped_words = set()
        
        # looping through each word in vocab
        for word in unique_words:
            # mapping only if ratio > 80
            try:
                if fuzz.ratio(profane,word) > 80:
                    mapped_words.add(word)
            except:
                pass
                
        # list of all vocab words for given profane word
        mapping_dict[profane] = mapped_words
    
    return mapping_dict

In [ ]:
def replaceWords(corpus, mapping_dict):
    processed_corpus = []

    for document in tqdm(corpus):

        # words = document.split()

        for mapped_word, v in mapping_dict.items():
            
            document = re.sub(r'\b{word}\b'.format(word = v), mapped_word, document)

            # for target_word in v:

            #     for i, word in enumerate(words):
            #         if word == target_word:
            #             words[i] = mapped_word

        # document = " ".join(words)
        document = document.strip()

        processed_corpus.append(document)

    return processed_corpus



In [ ]:
def final_processing(corpus):
    '''
    Function applies final processing steps post profane mapping such as removing special characters,
    punctuations etc.
    '''
    processed_comments = []
    stopwords_list = stopwords.words("english")
    # stopwords_list = nltk.download('stopwords')
    print('final_processing')

    emoj = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)


    # looping through each comment in corpus
    for comment in tqdm(corpus):
        try:
          processed_words = []
          comment = re.sub(emoj, r'', comment)
          comment = re.sub(r'http\S+', '', comment)
          # comment = re.sub("\B\#\w+", ' ', comment)
          # comment = re.sub("\B\@\w+", '', comment)
          comment = re.sub(r'(\w*#\w+|\w+#\w*)','', comment)
          comment = re.sub(r'(\w*@\w+|\w+#\w*)','',  comment)
          comment = re.sub(r'[^A-Za-z\s]+',"",  comment) # retain only letters
          for word in comment.split():
              if len(word) >= 3:
                  processed_words.append(word)
          
          comment = " ".join([e for e in processed_words if e.lower() not in stopwords_list])
          processed_comments.append(comment.strip())
        except Exception as e:
          print(corpus)
          print(e)
          pass
    
    return processed_comments

In [ ]:
def cleanComments(comments):
    processed_comments = []
    for comment in comments:
        comment = decontracted(comment)
        comment = removeNonPrintable(comment)

        # Lower comment
        processed_comments.append(comment.lower().strip())
      
    profane_words = getProfaneWords()
    unique_words = getUniqueWords(processed_comments)
    profane_dict = createMappingDict(profane_words, unique_words)
    processed_comments = replaceWords(processed_comments, profane_dict)
    final_comments = final_processing(processed_comments)
    return final_comments

In [ ]:
def corrHeatmap(df, fileName = None):
    classes = df.columns[1:]
    data = df.copy()
    data = data[classes]
    corr = data.corr()

    plt.figure(figsize=(8,6))
    sns.heatmap(corr,annot=True,vmin=1,vmax=0,fmt='.2g',cmap='rocket')
    plt.title("Correlation Matrix: Labels of Comments")
    if fileName:
        plt.savefig(fileName + '.png')
    plt.show()

In [ ]:
def showPercentiles(data):
    print("========== For 0-100 ==========")
    for i in range(11):
        print(f'{i*10}th Percentile Value = {np.percentile(data, i*10)}')

    print("\n")
    print("========== For 90-100 ==========")
    for i in range(11):
        print(f'{90+i}th Percentile Value = {np.percentile(data, 90 + i)}')

In [ ]:
def plotHist(data, bin_size, title, fileName = None, columns = None):
    if columns is not None:
        fig, axis = plt.subplots(len(columns)-1, 1, figsize = (12,30))
        for i, col in enumerate(columns[1:]):
            sns.histplot(data=data[col],bins=bin_size,palette="rocket",ax = axis[i])

        fig.tight_layout()
        fig.subplots_adjust(top = 0.95)
        fig.suptitle(title, size = 18)
    else:
        sns.histplot(data=data,bins=bin_size,palette="rocket")
        plt.title(title)
    

    if fileName:
        plt.savefig(fileName + '.png')
    
    plt.show()

In [ ]:
df_train.shape

In [ ]:
df_train.head()

In [ ]:
plotHist(df_train, 30, 'Histogram of the Labels', 'categories_histogram', df_train.columns)

In [ ]:
plotHist(df_train.Comment.str.len(), 50, 'Length of the Comments', 'comments_length_before_preprocess')

In [ ]:
for cat in df_train.columns[1:]:
    print(f"============== {cat} ==============")
    showPercentiles(df_train[cat])
    print("\n")

In [ ]:
showPercentiles(df_train.Comment.str.len())

In [ ]:
corrHeatmap(df_train, 'corr_matrix_labels')

There is a strong positive relationship between Toxicity-Severe Toxicity, Identity Attack-Insult and Toxicity-Profanity respectively

In [ ]:
processed_comments = cleanComments(df_train.Comment.values)

In [ ]:
df_processed = df_train[df_train.columns[1:]].copy()
df_processed['ProcessedComment'] = processed_comments

In [ ]:
cols = df_processed.columns.tolist()
ordered_cols = cols[-1:] + cols[:-1]
df_processed = df_processed[ordered_cols]

In [ ]:
df_processed.head()

In [ ]:
df_processed.shape

In [ ]:
plotHist(df_processed.ProcessedComment.str.len(), 50, 'Comments Length After Preprocessing', 'after_preprocess_comments_histogram')

As seen from the histogram above, the length most of the comments have is in between 0-300. Additionally, the histogram fits to logarithmic distribution and it is right-skewed. To examine the histogram furher, we can look into percentiles of the histogram to determine the vector dimensions which will be crucial for our deep learning model

In [ ]:
showPercentiles(df_processed.ProcessedComment.str.len())

Almost 95% of the comments have 1000 characters approximately. As a result, the input dimension of the tokenized vectors can be 1000.

# Model Building

Turn dataset into list(list of tokens, scores x6))

In [ ]:
VOCAB_SIZE = 1000
tokenizer = keras.preprocessing.text.Tokenizer(num_words=VOCAB_SIZE+1)

In [ ]:
labels_list = df_processed.dropna().loc[:,"Toxicity": "Threat"].convert_dtypes(infer_objects=True).__array__()[:,np.newaxis,:]

In [ ]:
df_processed.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_processed['ProcessedComment'].values, labels_list, train_size = 0.9)

In [ ]:
y_train = y_train[:,0,:]
y_test = y_test[:,0,:]


In [ ]:
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = pad_sequences(sequences=X_train, maxlen=VOCAB_SIZE, padding='post', truncating='post')
X_test = pad_sequences(sequences=X_test, maxlen=VOCAB_SIZE, padding='post', truncating='post')

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [ ]:
X_train.shape

In [ ]:
import pickle
# saving the training, validation and test tokenized data
fp = "tokenized_data.pkl"
with open(fp,mode="wb") as f:
    pickle.dump(obj=(X_train,
                     y_train,
                     X_test,
                     y_test),
                file=f)

In [ ]:
def get_sequential_architecture():
  model = keras.Sequential()

  model.add(layers.Dense(64, input_dim=VOCAB_SIZE, kernel_initializer='he_uniform', activation='relu'))
  model.add(layers.Dense(32, kernel_initializer='he_uniform'))
  model.add(layers.Dense(6))
  return model


In [ ]:
model_seq = get_sequential_architecture()
model_seq.compile(loss = keras.losses.BinaryCrossentropy(from_logits = True),
          optimizer = keras.optimizers.adam_v2.Adam(1e-4))

In [ ]:
model_seq.fit(x=X_train,
					y=y_train,
					epochs = 100,
					steps_per_epoch = math.floor(len(X_train)/100),
					verbose=1,
					validation_data=(X_test, y_test))

In [ ]:
def get_lstm_architecture(max_length,vocab_size,embedding_matrix):
    '''
    Function creates LSTM architecture with the input embedding matrix specified 
    '''

    # clearing backend session
    tf.keras.backend.clear_session()

    # defining input and embedding layers
    input_layer = Input(shape=(max_length,))
    # embedding = Embedding(input_dim=vocab_size,output_dim=300,input_length=max_length,weights=[embedding_matrix],trainable=False)(input_layer) 
    embedding = Embedding(input_dim=vocab_size,output_dim=300,input_length=max_length, trainable=True)(input_layer) 

    # bi-directional lstm layers
    lstm_output_1 = Bidirectional(LSTM(units=64,return_sequences=True))(embedding)
    drop = Dropout(rate=0.5)(lstm_output_1)
    lstm_output_2 = Bidirectional(LSTM(units=64,return_sequences=False))(drop)

    # output layer
    output_layer = Dense(units=6,activation='sigmoid')(lstm_output_2)

    # creating the model
    model = Model(inputs=input_layer,outputs=output_layer)

    return model

In [ ]:
model_lstm = get_lstm_architecture(VOCAB_SIZE, VOCAB_SIZE+1, None)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_lstm.compile(optimizer=optimizer,
              loss='binary_crossentropy', metrics = ['MAE'])

In [ ]:
reduced_lr = ReduceLROnPlateau(monitor="val_loss",
                               patience=1,
                               verbose=1)
early_stop = EarlyStopping(monitor="val_loss",
                           patience=2,
                           verbose=1)

In [ ]:
history = model_lstm.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = (X_test[:-5000], y_test[:-5000]))

In [ ]:
model_lstm.save("./model_lstm")

In [ ]:
y_pred = model_lstm.predict(X_test[-5000:])

In [ ]:
model_lstm.history

In [ ]:
test_loss, test_acc = model.evaluate(x=test_input, y=test_output)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)